**Problem 1**

In [1]:
import numpy as np
from scipy import linalg
import pandas as pd

In [2]:
def drazin(A, k, AD):
    if np.allclose(A @ AD, AD @ A) and \
    np.allclose(np.linalg.matrix_power(A, k + 1)@ AD,
    np.linalg.matrix_power(A, k)) and \
    np.allclose(AD @ A @ AD, AD):
        return True
    else:
        return False

In [3]:
# Test

A = np.array([[1, 3, 0, 0], [0, 1, 3, 0], [0, 0, 1, 3], [0, 0, 0, 0]])
AD = np.array([[1, -3, 9, 81], [0, 1,- 3, -18], [0, 0, 1, 3], [0, 0, 0, 0]])
k_A = 1

B = np.array([[1, 1, 3], [5, 2, 6], [-2, -1, -3]])
BD = np.zeros_like(B)
k_B = 3

print(drazin(A,k_A,AD))
print(drazin(B,k_B,BD))

True
True


**Problem 2**

In [4]:
def drazin2(A, tol):
    n, n = A.shape
    f = lambda x: abs(x) > tol
    g = lambda x: abs(x) <= tol
    Q1, S, k1 = linalg.schur(A, sort=f)
    Q2, T, k2 = linalg.schur(A, sort=g)
    U = np.concatenate((S[:, :k1], T[:, :n - k1]), axis = 1)
    Uinv = linalg.inv(U)
    V = Uinv @ A @ U
    Z = np.zeros((n,n))
    if k1 != 0:
        Minv = linalg.inv(V[:k1, :k1])
        Z[:k1, :k1] = Minv
    return U @ Z @ Uinv

In [5]:
# Verification

AD = drazin2(A, 1e-5) # Use A from problem 1
print(AD)
print(drazin(A, 1, AD))

BD = drazin2(B, 1e-5) # Use B from problem 2
print(BD)
print(drazin(B, 3, BD))

[[  1.  -3.   9.  81.]
 [  0.   1.  -3. -18.]
 [  0.   0.   1.   3.]
 [  0.   0.   0.   0.]]
True
[[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
True


**Problem 3**

In [6]:
def effect_res(A):
    deg = np.sum(A, 0)
    Diag = np.diag(deg)
    L = Diag - A
    
    n = np.shape(A)[0]
    I = np.eye(n)
    Z = np.zeros((n, n))
    for i in range(0, n):
        Li = np.copy(L)
        Li[i, :] = I[i, :]
        LiDiag = drazin2(Li, 0.001)
        for j in range(0, n):
            if j != i:
                Z[j, i] = LiDiag[j, j]
    return Z

In [7]:
# Test

A = np.array(([0, 1, 0, 0], [1, 0, 1, 0], [0, 1, 0, 1], [0, 0, 1, 0]))
RA = effect_res(A)
print(np.allclose(3, RA[0, 3]))

B = np.array(([0, 1, 1], [1, 0, 1], [1, 1, 0]))
RB = effect_res(B)
print(np.allclose(2 / 3, RB[0, 1]))

C = np.array(([0, 2], [2, 0]))
RC = effect_res(C)
print(np.allclose(1 / 2, RC[0, 1]))

D = np.array(([0, 1], [1, 0]))
RD = effect_res(D)
print(np.allclose(1, RD[0, 1]))

E = np.array(([0, 3], [3, 0]))
RE = effect_res(E)
print(np.allclose(1 / 3, RE[0, 1]))

F = np.array(([0, 4], [4, 0]))
RF = effect_res(F)
print(np.allclose(1 / 4, RF[0, 1]))

True
True
True
True
True
True


**Problem 4 and 5**

In [8]:
class LinkPredictor:
    
    def __init__(self, file):
        
        network = pd.read_csv(file, header=None).as_matrix()
        m, n = network.shape
        
        nodes, index = np.unique(network, return_inverse=True)
        self.nodes = nodes
        
        index = index.reshape((m, n))
        self.len = len(nodes)
        Adj_mat = np.zeros((self.len, self.len))
        
        for i in range(m):
            Adj_mat[index[i, 0], index[i, 1]] = 1
            Adj_mat[index[i, 1], index[i, 0]] = 1
        self.Adj_mat = Adj_mat
        
        R = effect_res(Adj_mat)
        self.R = R
        self.max_R = R.max()
        
    def predict_link(self, node=None):
        if node == None:
            Adj_copy = np.copy(self.Adj_mat)
            R_copy = np.copy(self.R)
            new = R_copy + self.max_R * Adj_copy + self.max_R * \
            np.eye(self.len)
            index_min = np.unravel_index(new.argmin(), new.shape)
            return(self.nodes[index_min[0]], self.nodes[index_min[1]])
        
        else:
            if node in self.nodes:
                index = np.where(self.nodes == node)[0][0]
                Adj_copy = np.copy(self.Adj_mat)
                R_copy = np.copy(self.R)
                new = R_copy + Adj_copy + np.eye(self.len)
                new = new[:, index]
                index_min = np.unravel_index(new.argmin(), new.shape)
                return self.nodes[index_min[0]]
                
            else:
                raise ValueError("Node is not in the network.")
                
    def add_link(self, node1, node2):
        
        if node1 in self.nodes and node2 in self.nodes:
            index1 = np.where(self.nodes == node1)[0][0]
            index2 = np.where(self.nodes == node2)[0][0]
            
            self.Adj_mat[index1,index2] = self.Adj_mat[index1,index2] + 1
            self.Adj_mat[index2,index1] = self.Adj_mat[index2,index1] + 1
            self.R = effect_res(self.Adj_mat)
        else:
            raise ValueError("Either node1 or node2 is not in the network.")

In [9]:
net = LinkPredictor('social_network.csv')
net.predict_link()

C:\Users\danie\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:5: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """


('Oliver', 'Emily')

In [10]:
net.predict_link('Melanie')

'Carol'

In [11]:
net.predict_link('Alan')

'Sonia'

In [12]:
net.add_link('Alan', 'Sonia')
net.predict_link('Alan')

'Piers'

In [13]:
net.add_link('Alan', 'Piers')
net.predict_link('Alan')

'Abigail'